### Course lecture
[1 - Introduction to Random Forests](http://course18.fast.ai/lessonsml1/lesson1.html)

In [8]:
import numpy as np
import pandas as pd

from collections import OrderedDict
from pandas.api.types import is_numeric_dtype
from typing import Any, Iterable

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)

In [2]:
PATH = '../data/'

In [3]:
df_raw = pd.read_csv(f'{PATH}Train.zip', low_memory=False, parse_dates=['saledate'])

In [4]:
# Smaller DF to test algorithms
#dft = df_raw.iloc[0:50].copy(deep=True)

### Inspect the data

In [15]:
df_raw.iloc[0:50][['saledate', 'UsageBand', 'Forks']].to_csv('testdata.csv')

In [12]:
df_raw.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,...,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,...,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,...,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,...,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,...,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,...,NaN,NaN,NaN,NaN,NaN


In [6]:
def inspect_df(df: pd.DataFrame, display_stats: bool=False) -> pd.DataFrame:
    """ Show column types and null values in DataFrame df
    """
    if len(df) == 0: 
        print('Empty DataFrame')
        return
    
    resdict = OrderedDict()
    
    # Inspect nulls
    null_series = df.isnull().sum()
    resdict['column'] = null_series.index
    resdict['null_fraction'] = np.round(null_series.values / len(df), 3)
    resdict['nulls'] = null_series.values
    # Inspect types
    types = df.dtypes.values
    type_names = [t.name for t in types]
    resdict['type'] = type_names
    # Is numeric?
    is_numeric = []
    for col in df.columns:
        is_numeric.append(is_numeric_dtype(df[col]))
    resdict['is_numeric'] = is_numeric
    # Dataframe
    resdf = pd.DataFrame(resdict)
    resdf.sort_values('null_fraction', inplace=True)
    resdf.reset_index(inplace=True, drop=True)
    
    # Choose action
    if display_stats:
        with pd.option_context('display.max_rows', len(resdf)):
            display(resdf)

    return resdf

In [7]:
def summarize_df(df: pd.DataFrame, display_stats: bool=False) -> pd.DataFrame:
    """ Show stats;
        - rows:
            - column types
        - columns
            - number of columns
            - number of cols containing NaN's
    
    """
    # Original DataFrame
    (nrows, ncols) = df.shape
    # Stats of DataFrame
    stats = inspect_df(df, display_stats=False)
    data_types = np.unique(stats['type'].values)
    
    resdict = OrderedDict()

    # Column: data types
    resdict['type'] = data_types
    
    ncols_type = []
    ncols_nan = []
    n_nans = []
    n_total = []
    
    for dt in data_types:
        # Column: number of columns with type
        nc = len(stats[ stats['type'] == dt ]) 
        ncols_type.append(nc)
        
        # Column: number of columns with NaNs
        nan_cols = stats[ (stats['type'] == dt) & stats['nulls'] > 0]
        ncols_nan.append(len(nan_cols))
        
        # Column: number of NaNs
        n_nans.append(nan_cols['nulls'].sum())
        
        # Column: total number of values
        n_total.append(nc*nrows)
    
    # Prepare dict for the df
    resdict['ncols'] = ncols_type
    resdict['ncols_w_nans'] = ncols_nan
    resdict['n_nans'] = n_nans
    resdict['n_total'] = n_total
    
    # Proportions of NaNs in each column group.
    # Division by zero shouldn't occur
    nan_frac = np.array(n_nans) / np.array(n_total)
    resdict['nan_frac'] = np.round(nan_frac, 2)
    
    resdf = pd.DataFrame(resdict)
    resdf.sort_values('type', inplace=True)
    resdf.reset_index(inplace=True, drop=True)

    # Choose action
    if display_stats:
        with pd.option_context('display.max_rows', len(resdf)):
            display(resdf)
    
    return resdf

In [74]:
#_ = inspect_df(df_raw, display_stats=True)
_ = summarize_df(df_raw, display_stats=True)

,type,ncols,ncols_w_nans,n_nans,n_total,nan_frac
0,datetime64[ns],1,0,0,401125,0.00
1,float64,2,0,0,802250,0.00
2,int64,6,0,0,2406750,0.00
3,object,44,22,6002766,17649500,0.34


### Add log price

In [75]:
df_proc = df_raw.copy(deep=True)
df_proc['LogSalePrice'] = np.log(df_proc['SalePrice'])
df_proc.drop(columns=['SalePrice'], inplace=True)

In [76]:
df_proc.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,...,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,LogSalePrice
0,1139246,999089,3157,121,3.0,...,NaN,NaN,Standard,Conventional,11.097410
1,1139248,117657,77,121,3.0,...,NaN,NaN,Standard,Conventional,10.950807
2,1139249,434808,7009,121,3.0,...,NaN,NaN,NaN,NaN,9.210340
3,1139251,1026470,332,121,3.0,...,NaN,NaN,NaN,NaN,10.558414
4,1139253,1057373,17311,121,3.0,...,NaN,NaN,NaN,NaN,9.305651


### Extract date properties
See [Attributes of Pandas Timestamp](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html)

In [77]:
def add_datefields(df: pd.DataFrame, column: str, drop_original: bool=False, 
                   inplace: bool=False, attrs: Iterable[Any]=None) -> pd.DataFrame:
    """ Add attributes of the date to dataFrame df
    """
    raw_date = df[column]
    
    # Pandas datetime attributes
    if attrs is None:
        attributes = ['dayofweek', 
            'dayofyear', 
            'is_month_end', 
             'is_month_start', 
             'is_quarter_end', 
             'is_quarter_start', 
             'quarter', 
             'week']
    else:
        attributes = attrs
    
    # Return new?
    if inplace:
        resdf = df
    else:
        resdf = df.copy(deep=True)
    
    # Could probably be optimized with pd.apply()
    for attr in attributes:
        new_column = f'{column}_{attr}'
        # https://stackoverflow.com/questions/2612610/
        new_vals = [getattr(d, attr) for d in raw_date]
        resdf[new_column] = new_vals
        
    if drop_original:
        resdf.drop(columns=column, inplace=True)
    
    return resdf

In [78]:
if 'saledate' in df_proc.columns:
    df_proc = add_datefields(df_raw, 'saledate', drop_original=True)

In [79]:
df_proc.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,...,saledate_is_month_start,saledate_is_quarter_end,saledate_is_quarter_start,saledate_quarter,saledate_week
0,1139246,66000,999089,3157,121,...,False,False,False,4,46
1,1139248,57000,117657,77,121,...,False,False,False,1,13
2,1139249,10000,434808,7009,121,...,False,False,False,1,9
3,1139251,38500,1026470,332,121,...,False,False,False,2,20
4,1139253,11000,1057373,17311,121,...,False,False,False,3,30


### Add NaN column
Here we add a columns indicating whether the original column contained a NaN

In [80]:
def add_nan_columns(df: pd.DataFrame, inplace: bool=False, column_list: Iterable[Any]=None) -> pd.DataFrame:
    """ For each column containing NaNs, add a boolean
        column specifying if the column is NaN. Can be used
        if the data is later imputated.
    """
    if column_list is not None:
        nan_columns = column_list
    else:
        # Get names of columns containing at least one NaN
        temp = (df.isnull().sum() != 0)
        nan_columns = temp.index[temp.values]
    
    # Return new?
    if inplace:
        resdf = df
    else:
        resdf = df.copy(deep=True)
    
    for column in nan_columns:
        new_column = f'{column}_isnull'
        nans = df[column].isnull()
        resdf[new_column] = nans

    return resdf

In [81]:
df_proc = add_nan_columns(df_proc)
df_proc.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,...,Backhoe_Mounting_isnull,Blade_Type_isnull,Travel_Controls_isnull,Differential_Type_isnull,Steering_Controls_isnull
0,1139246,66000,999089,3157,121,...,True,True,True,False,False
1,1139248,57000,117657,77,121,...,True,True,True,False,False
2,1139249,10000,434808,7009,121,...,True,True,True,True,True
3,1139251,38500,1026470,332,121,...,True,True,True,True,True
4,1139253,11000,1057373,17311,121,...,True,True,True,True,True


In [82]:
#inspect_df(tmp)

### Handle numerical types containing NaN

In [83]:
def numeric_nans(df: pd.DataFrame, display_stats: bool=False) -> pd.DataFrame:
    """ Inspect numerical NaN values of a DataFrame df
    """
    stats = inspect_df(df)
    nan_stats = stats.loc[ (stats['is_numeric'] == True) & (stats['nulls'] >0) ].copy(deep=True)
    
    len_uniques = []
    uniques = []
    
    for row in nan_stats['column'].values:
        uniq = np.unique(df[row][ df[row].notnull() ].values)
        len_uniques.append(len(uniq))
        uniques.append(uniq)
    
    nan_stats['num_uniques'] = len_uniques
    nan_stats['uniques'] = uniques
    nan_stats.reset_index(inplace=True, drop=True)
    
    if display_stats:
        with pd.option_context('display.max_rows', len(nan_stats)):
            display(nan_stats)
    
    return nan_stats

In [84]:
stats = numeric_nans(df_raw)
stats

,column,null_fraction,nulls,type,is_numeric,num_uniques,uniques
0,auctioneerID,0.050,20136,float64,True,30,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,MachineHoursCurrentMeter,0.644,258360,float64,True,15152,"[0.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ..."


In [85]:
#stats.loc[ stats['column'] == 'auctioneerID'].uniques.values

In [86]:
df_proc['auctioneerID'] = df_proc['auctioneerID'].fillna(
    df_proc['auctioneerID'].max() + 1
)

df_proc['MachineHoursCurrentMeter'] = df_proc['MachineHoursCurrentMeter'].fillna(
    df_proc['MachineHoursCurrentMeter'].median()
)

In [87]:
numeric_nans(df_proc)

,column,null_fraction,nulls,type,is_numeric,num_uniques,uniques


### Add categories
- http://benalexkeen.com/mapping-categorical-data-in-pandas/
- https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html

In [102]:
def categorize_df(df: pd.DataFrame, columns: Iterable[Any]=None, 
                  inplace: bool=False, drop_original: bool=True) -> (pd.DataFrame, pd.DataFrame):
    """ Categorize values in columns, and replace value with category.
        If no columns are given, default to all 'object' columns
    """
    if columns is not None:
        cat_cols = columns
    else:
        cat_cols = df.columns[ [dt.name == 'object' for dt in df.dtypes.values] ]
    
    if inplace:
        resdf = df
    else:
        resdf = df.copy(deep=True)
    
    df_codes = []
    df_cats = []
    n_categories = []
    
    for column in cat_cols:
        new_column = f'{column}_category'
        cat_column = df[column].astype('category')
        # By default, NaN is -1. We convert to zero by incrementing all.
        col_codes = cat_column.cat.codes + 1
        resdf[new_column] = col_codes
        
        # DataFrame with the codes
        df_codes.append(col_codes)
        df_cats.append(cat_column.cat.categories)
        n_categories.append(len(np.unique(col_codes)))
    
    cat_dict = OrderedDict()
    cat_dict['column'] = cat_cols
    cat_dict['n_categories'] = n_categories
    cat_dict['categories'] = df_cats
    cat_dict['codes'] = df_codes
    cat_df = pd.DataFrame(cat_dict)
    
    if drop_original:
        resdf.drop(columns=cat_cols, inplace=True)
    
    return (resdf, cat_df)

In [103]:
(df_cats, catcodes) = categorize_df(df_proc)

In [104]:
df_cats.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,...,Backhoe_Mounting_category,Blade_Type_category,Travel_Controls_category,Differential_Type_category,Steering_Controls_category
0,1139246,66000,999089,3157,121,...,0,0,0,4,2
1,1139248,57000,117657,77,121,...,0,0,0,4,2
2,1139249,10000,434808,7009,121,...,0,0,0,0,0
3,1139251,38500,1026470,332,121,...,0,0,0,0,0
4,1139253,11000,1057373,17311,121,...,0,0,0,0,0


In [119]:
catcodes

,column,n_categories,categories,codes
0,UsageBand,4,"Index(['High', 'Low', 'Medium'], dtype='object')",0 2 1 2 2 1 3 ...
1,fiModelDesc,4999,"Index(['100C', '104', '1066', '1066E', '1080',...",0 950 1 1725 2 331 3...
2,fiBaseModel,1950,"Index(['10', '100', '104', '1066', '1080', '10...",0 296 1 527 2 110 3...
3,fiSecondaryDesc,176,"Index([' MSR SPIN ACE', '#NAME?', '-2', '-3', ...",0 41 1 55 2 0 3 ...
4,fiModelSeries,123,"Index([' III', '#NAME?', '-1', '-1.50E+01', '-...",0 0 1 98 2 0 3 ...
...,...,...,...,...
39,Backhoe_Mounting,3,"Index(['None or Unspecified', 'Yes'], dtype='o...",0 0 1 0 2 0 3 ...
40,Blade_Type,11,"Index(['Angle', 'Coal', 'Landfill', 'No', 'Non...",0 0 1 0 2 0 3 ...
41,Travel_Controls,8,"Index(['1 Speed', '2 Pedal', 'Differential Ste...",0 0 1 0 2 0 3 ...
42,Differential_Type,5,"Index(['Limited Slip', 'Locking', 'No Spin', '...",0 4 1 4 2 0 3 ...


### Inspect results of data processing

In [106]:
summarize_df(df_raw)

,type,ncols,ncols_w_nans,n_nans,n_total,nan_frac
0,datetime64[ns],1,0,0,401125,0.00
1,float64,2,0,0,802250,0.00
2,int64,6,0,0,2406750,0.00
3,object,44,22,6002766,17649500,0.34


In [109]:
summarize_df(df_cats)

,type,ncols,ncols_w_nans,n_nans,n_total,nan_frac
0,bool,44,0,0,17649500,0.0
1,float64,2,0,0,802250,0.0
2,int16,4,0,0,1604500,0.0
3,int64,10,0,0,4011250,0.0
4,int8,40,0,0,16045000,0.0


In [110]:
#_ = inspect_df(df_proc, display_stats=True)

In [111]:
df_cats.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,...,Backhoe_Mounting_category,Blade_Type_category,Travel_Controls_category,Differential_Type_category,Steering_Controls_category
0,1139246,66000,999089,3157,121,...,0,0,0,4,2
1,1139248,57000,117657,77,121,...,0,0,0,4,2
2,1139249,10000,434808,7009,121,...,0,0,0,0,0
3,1139251,38500,1026470,332,121,...,0,0,0,0,0
4,1139253,11000,1057373,17311,121,...,0,0,0,0,0


### Split into train and test

In [113]:
dfc = df_cats.iloc[0:50].copy(deep=True)

(nrows, ncols) = df_cats.shape
print('Data rows: {}'.format(nrows))

Data rows: 401125


In [29]:
#df_proc.iloc[[1,2,100]]

In [114]:
np.random.seed(seed=23)
x = dfc.index.values
x

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [115]:
x = np.random.permutation(x)
x

array([14, 18, 17, 32, 13, 33, 20,  8, 36, 28, 29, 10, 44,  3, 22, 48, 30,
       35, 24, 23,  4, 42,  7,  1, 49, 41, 45, 15, 46, 16, 34, 37,  0,  5,
       21, 11,  2, 43,  6, 25, 26, 39, 27, 12, 31, 47,  9, 40, 38, 19])

In [116]:
N = 10
y = np.sort(x[:10])
y

array([ 8, 13, 14, 17, 18, 20, 28, 32, 33, 36])

In [118]:
dfc.iloc[y]

,SalesID,SalePrice,MachineID,ModelID,datasource,...,Backhoe_Mounting_category,Blade_Type_category,Travel_Controls_category,Differential_Type_category,Steering_Controls_category
8,1139272,21500,1036251,36003,121,...,0,0,0,0,0
13,1139284,30500,1068082,3542,121,...,0,0,0,0,0
14,1139290,28000,1058450,5162,121,...,0,0,0,0,0
17,1139299,9500,1002713,21442,121,...,0,0,0,0,0
18,1139301,12500,125790,7040,121,...,0,0,0,0,0
20,1139311,41000,1014135,8867,121,...,0,0,0,0,0
28,1139357,46000,44800,19167,121,...,0,0,0,0,0
32,1139367,31500,1036100,9109,121,...,0,0,0,4,2
33,1139369,14000,1050658,1918,121,...,1,5,6,0,0
36,1139390,8250,1011114,7110,121,...,0,0,0,0,0
